# This notebook was created working along to Effective Pandas chapter 30 by Matt Harrison

In [1]:
import pandas as pd

In [2]:
scores = pd.DataFrame({
    'name':['Adam', 'Bob', 'Dave', 'Fred'],
    'age': [15, 16, 16, 15],
    'test1': [95, 82, 89, None],
    'test2': [80, 82, 84, 88],
    'teacher': ['Ashby', 'Ashby', 'Jones', 'Jones']})

In [3]:
scores

,name,age,test1,test2,teacher
0,Adam,15,95.0,80,Ashby
1,Bob,16,82.0,82,Ashby
2,Dave,16,89.0,84,Jones
3,Fred,15,NaN,88,Jones


In [4]:
scores.melt(id_vars=['name', 'age'],
           value_vars=['test1', 'test2'])

,name,age,variable,value
0,Adam,15,test1,95.0
1,Bob,16,test1,82.0
2,Dave,16,test1,89.0
3,Fred,15,test1,NaN
4,Adam,15,test2,80.0
5,Bob,16,test2,82.0
6,Dave,16,test2,84.0
7,Fred,15,test2,88.0


In [5]:
(scores
    .groupby(['name', 'age'])
    .apply(lambda g: pd.concat([
        g[['test1']].rename(columns={'test1':'val'}).assign(var='test1'),
        g[['test2']].rename(columns={'test2':'val'}).assign(var='test2')]))
     .reset_index()
     .drop(columns='level_2')
)

,name,age,val,var
0,Adam,15,95.0,test1
1,Adam,15,80.0,test2
2,Bob,16,82.0,test1
3,Bob,16,82.0,test2
4,Dave,16,89.0,test1
5,Dave,16,84.0,test2
6,Fred,15,NaN,test1
7,Fred,15,88.0,test2


In [6]:
scores.melt(id_vars=['name', 'age'],
           value_vars=['test1', 'test2'],
           var_name='test', value_name='score')

,name,age,test,score
0,Adam,15,test1,95.0
1,Bob,16,test1,82.0
2,Dave,16,test1,89.0
3,Fred,15,test1,NaN
4,Adam,15,test2,80.0
5,Bob,16,test2,82.0
6,Dave,16,test2,84.0
7,Fred,15,test2,88.0


In [7]:
scores.melt(id_vars=['name', 'age', 'teacher'],
           value_vars=['test1', 'test2'],
           var_name='test', value_name='score')

,name,age,teacher,test,score
0,Adam,15,Ashby,test1,95.0
1,Bob,16,Ashby,test1,82.0
2,Dave,16,Jones,test1,89.0
3,Fred,15,Jones,test1,NaN
4,Adam,15,Ashby,test2,80.0
5,Bob,16,Ashby,test2,82.0
6,Dave,16,Jones,test2,84.0
7,Fred,15,Jones,test2,88.0


In [8]:
melted = scores.melt(id_vars=['name', 'age', 'teacher'],
                    value_vars=['test1', 'test2'],
                    var_name='test', value_name='score')

In [9]:
melted

,name,age,teacher,test,score
0,Adam,15,Ashby,test1,95.0
1,Bob,16,Ashby,test1,82.0
2,Dave,16,Jones,test1,89.0
3,Fred,15,Jones,test1,NaN
4,Adam,15,Ashby,test2,80.0
5,Bob,16,Ashby,test2,82.0
6,Dave,16,Jones,test2,84.0
7,Fred,15,Jones,test2,88.0


In [10]:
(melted
    .pivot_table(index=['name', 'age', 'teacher'],
                columns='test', values='score')
    .reset_index()
)

test,name,age,teacher,test1,test2
0,Adam,15,Ashby,95.0,80.0
1,Bob,16,Ashby,82.0,82.0
2,Dave,16,Jones,89.0,84.0
3,Fred,15,Jones,NaN,88.0


In [11]:
(melted
    .groupby(['name', 'age', 'teacher', 'test'])
    .score
    .mean()
    .unstack()
    .reset_index()
)

test,name,age,teacher,test1,test2
0,Adam,15,Ashby,95.0,80.0
1,Bob,16,Ashby,82.0,82.0
2,Dave,16,Jones,89.0,84.0
3,Fred,15,Jones,NaN,88.0


In [12]:
def get_uniq_cols(jb):
    counter = collections.defaultdict(list)
    for col in sorted(jb.columns):
        period_count = col.count('.')
        if period_count >= 2:
            part_end = 2
        else:
            part_end = 1
        parts = col.split('.')[:part_end]
        counter['.'.join(parts)].append(col)
    uniq_cols = []
    for cols in counter.values():
        if len(cols) == 1:
            uniq_cols.extend(cols)
    return uniq_cols

In [13]:
def prep_for_ml(df):
    # remove pandas types
    return (df
     .assign(**{col:df[col].astype(float) 
               for col in df.select_dtypes('number')},
             **{col:df[col].astype(str).fillna('') 
               for col in df.select_dtypes(['object', 'category'])})
    )

In [14]:
def predict_col(df, col):
    df = prep_for_ml(df)
    missing = df.query(f'~{col}.isna()')
    cat_idx = []
    for i,typ in enumerate(df.drop(columns=[col]).dtypes):
        if str(typ) == 'object':
            cat_idx.append(i)
    X = (missing
         .drop(columns=[col])
         .values
        )
    y = missing[col]
    model = cb.CatBoostRegressor(iterations=20, cat_features=cat_idx)
    model.fit(X, y, cat_features=cat_idx)
    pred = model.predict(df.drop(columns=[col]))
    return df[col].where(~df[col].isna(), pred)

In [15]:
import numpy as np
import collections
import catboost as cb

In [16]:
def tweak_jb(jb):
     uniq_cols = get_uniq_cols(jb)
     return (jb
         [uniq_cols]
         .rename(columns=lambda c: c.replace('.', '_'))
         .assign(age=lambda df_:df_.age.str.slice(0,2).astype(float)
                    .astype('Int64'),
                are_you_datascientist=lambda df_:df_
                    .are_you_datascientist
                    .replace({'Yes': True, 'No': False, np.nan: False}),
                company_size=lambda df_:df_.company_size.replace({
                    'Just me': 1, 'Not sure': np.nan,
                    'More than 5,000': 5000, '2–10': 2, '11–50':11,
                    '51–500': 51, '501–1,000':501,
                    '1,001–5,000':1001}).astype('Int64'),
                country_live=lambda df_:df_.country_live
                    .astype('category'),
                employment_status=lambda df_:df_.employment_status
                     .fillna('Other').astype('category'),
                is_python_main=lambda df_:df_.is_python_main
                     .astype('category'),
                team_size=lambda df_:df_.team_size
                    .str.split(r'-', n=1, expand=True)
                    .iloc[:,0].replace('More than 40 people', 41)
                    .where(df_.company_size!=1, 1).astype(float),
                years_of_coding=lambda df_:df_.years_of_coding
                    .replace('Less than 1 year', .5)
                    .str.extract(r'(\d+)').astype(float),
                python_years=lambda df_:df_.python_years
                    .replace('Less than 1 year', .5)
                    .str.extract(r'(\d+)').astype(float),
                python3_ver=lambda df_:df_.python3_version_most
                     .str.replace('_', '.').str.extract(r'(\d\.\d)')
                     .astype(float),
                use_python_most=lambda df_:df_.use_python_most
                     .fillna('Unknown')
               )
        .assign(team_size=lambda df_:predict_col(df_, 'team_size')
             .astype(int))
        .drop(columns=['python2_version_most'])
        .dropna()
    )    
url = 'https://github.com/mattharrison/datasets/raw/master/data/'\
   '2020-jetbrains-python-survey.csv'
jb = pd.read_csv(url)
jb2 = tweak_jb(jb)

/tmp/ipykernel_48/2582374631.py:45: DtypeWarning: Columns (152) have mixed types. Specify dtype option on import or set low_memory=False.
  jb = pd.read_csv(url)


Learning rate set to 0.5
0:	learn: 2.9695218	total: 92.2ms	remaining: 1.75s
1:	learn: 2.8766539	total: 126ms	remaining: 1.13s
2:	learn: 2.8387189	total: 163ms	remaining: 926ms
3:	learn: 2.8028751	total: 194ms	remaining: 778ms
4:	learn: 2.7899957	total: 225ms	remaining: 674ms
5:	learn: 2.7749439	total: 256ms	remaining: 596ms
6:	learn: 2.7719128	total: 283ms	remaining: 526ms
7:	learn: 2.7649792	total: 313ms	remaining: 469ms
8:	learn: 2.7649588	total: 336ms	remaining: 411ms
9:	learn: 2.7630617	total: 364ms	remaining: 364ms
10:	learn: 2.7625779	total: 395ms	remaining: 323ms
11:	learn: 2.7515902	total: 424ms	remaining: 283ms
12:	learn: 2.7513459	total: 452ms	remaining: 244ms
13:	learn: 2.7445634	total: 481ms	remaining: 206ms
14:	learn: 2.7443257	total: 510ms	remaining: 170ms
15:	learn: 2.7423142	total: 537ms	remaining: 134ms
16:	learn: 2.7419144	total: 563ms	remaining: 99.4ms
17:	learn: 2.7399388	total: 591ms	remaining: 65.6ms
18:	learn: 2.7384297	total: 618ms	remaining: 32.5ms
19:	learn: 2

In [17]:
jb2

,age,are_you_datascientist,company_size,country_live,employment_status,first_learn_about_main_ide,how_often_use_main_ide,ide_main,is_python_main,job_team,main_purposes,missing_features_main_ide,nps_main_ide,python_years,python3_version_most,several_projects,team_size,use_python_most,years_of_coding,python3_ver
1,21,True,5000,India,Fully employed by a company / organization,School / University,Daily,VS Code,Yes,Work in a team,Both for work and personal,"No, it has all the features I need",8.0,3.0,Python 3_6,"Yes, I work on one main and several side projects",2,Software prototyping,3.0,3.6
2,30,False,5000,United States,Fully employed by a company / organization,Friend / Colleague,Daily,Vim,Yes,Work on your own project(s) independently,Both for work and personal,"No, it has all the features I need",10.0,3.0,Python 3_6,"Yes, I work on one main and several side projects",4,DevOps / System administration / Writing autom...,3.0,3.6
10,21,False,51,Other country,Fully employed by a company / organization,School / University,Daily,IntelliJ IDEA,Yes,Work in a team,Both for work and personal,"No, it has all the features I need",10.0,1.0,Python 3_8,"Yes, I work on one main and several side projects",2,Web development,1.0,3.8
11,21,True,51,United States,Fully employed by a company / organization,Online learning platform / Online course,Daily,PyCharm Community Edition,Yes,Work in a team,Both for work and personal,"No, it has all the features I need",9.0,3.0,Python 3_9,"Yes, I work on many different projects",2,Data analysis,3.0,3.9
13,30,True,5000,Belgium,Fully employed by a company / organization,Social network,Daily,VS Code,Yes,Work in a team,Both for work and personal,"No, it has all the features I need",10.0,6.0,Python 3_7,"Yes, I work on many different projects",2,Data analysis,3.0,3.7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54456,30,False,1001,Turkey,Fully employed by a company / organization,Friend / Colleague,Daily,PyCharm Community Edition,Yes,Work on your own project(s) independently,Both for work and personal,"No, it has all the features I need",9.0,1.0,Python 3_6,"Yes, I work on many different projects",5,Machine learning,6.0,3.6
54457,21,False,2,Russian Federation,Fully employed by a company / organization,School / University,Daily,Vim,Yes,Work on your own project(s) independently,Both for work and personal,"No, it has all the features I need",10.0,6.0,Python 3_6,"Yes, I work on many different projects",2,Data analysis,1.0,3.6
54459,21,False,1,Russian Federation,Self-employed (a person earning income directl...,Friend / Colleague,Daily,PyCharm Professional Edition,Yes,Work in a team,Both for work and personal,"No, it has all the features I need",10.0,3.0,Python 3_7,"Yes, I work on many different projects",1,Web development,6.0,3.7
54460,30,True,51,Spain,Fully employed by a company / organization,Search engines,Daily,Other,Yes,Work on your own project(s) independently,Both for work and personal,Yes – Please list:,3.0,6.0,Python 3_7,"Yes, I work on many different projects",4,Data analysis,3.0,3.7


In [18]:
(jb2
    .groupby(['country_live', 'are_you_datascientist'])
    .size()
)

TypeError: '<' not supported between instances of 'str' and 'bool'

In [19]:
(jb2
    .groupby(['country_live', 'are_you_datascientist'])
    .size()
    .unstack()
)

TypeError: '<' not supported between instances of 'str' and 'bool'

In [20]:
(jb2
    .groupby(['country_live', 'are_you_datascientist'])
    .size()
    .unstack(0)
)

TypeError: '<' not supported between instances of 'str' and 'bool'

In [21]:
(jb2
    .groupby(['country_live', 'are_you_datascientist'])
    .size()
    .unstack('country_live')
)

TypeError: '<' not supported between instances of 'str' and 'bool'

In [22]:
(jb2
    .pivot_table(index='country_live',
                aggfunc={'age':['min', 'max'],
                        'company_size':['min', 'max']})
)

age     company_size    
              max min          max min
country_live                          
Algeria        60  18         5000   1
Argentina      60  18         5000   1
Armenia        30  18         5000   1
Australia      60  18         5000   1
Austria        50  18         5000   1
...            ..  ..          ...  ..
United States  60  18         5000   1
Uruguay        40  21         5000   2
Uzbekistan     21  21         5000   1
Venezuela      50  18         5000   1
Viet Nam       60  18         5000   1

[76 rows x 4 columns]

In [23]:
(jb2
    .pivot_table(index='country_live',
                aggfunc={'age':['min', 'max'],
                        'company_size': ['min','max']})
    .stack(0)
)

max  min
country_live                        
Algeria      age             60   18
             company_size  5000    1
Argentina    age             60   18
             company_size  5000    1
Armenia      age             30   18
...                         ...  ...
Uzbekistan   company_size  5000    1
Venezuela    age             50   18
             company_size  5000    1
Viet Nam     age             60   18
             company_size  5000    1

[152 rows x 2 columns]

In [24]:
(jb2
    .pivot_table(index='country_live',
                aggfunc={'age':['min','max'],
                        'company_size':['min', 'max']})
    .stack(1)
)

age  company_size
country_live                       
Algeria      max   60          5000
             min   18             1
Argentina    max   60          5000
             min   18             1
Armenia      max   30          5000
...               ...           ...
Uzbekistan   min   21             1
Venezuela    max   50          5000
             min   18             1
Viet Nam     max   60          5000
             min   18             1

[152 rows x 2 columns]

In [25]:
(jb2
    .pivot_table(index='country_live',
                aggfunc={'age':['min', 'max'],
                        'company_size':['min','max']})
    .stack(1)
    .swaplevel()
)

,,age,company_size
,country_live,,
max,Algeria,60,5000
min,Algeria,18,1
max,Argentina,60,5000
min,Argentina,18,1
max,Armenia,30,5000
...,...,...,...
min,Uzbekistan,21,1
max,Venezuela,50,5000
min,Venezuela,18,1


In [27]:
(jb2
    .groupby(['country_live', 'age'])
    .mean()
)

TypeError: unsupported operand type(s) for +: 'int' and 'str'

In [28]:
(jb2
    .groupby(['country_live', 'age'])
    .mean()
    .reset_index()
)

TypeError: unsupported operand type(s) for +: 'int' and 'str'

In [29]:
(jb2
    .groupby(['country_live', 'age'], as_index=False)
    .mean()
)

TypeError: unsupported operand type(s) for +: 'int' and 'str'

In [30]:
(jb2
    .groupby(['country_live', 'age'])
    .mean()
    .unstack()
)

TypeError: unsupported operand type(s) for +: 'int' and 'str'

In [31]:
def flatten_cols(df):
    cols = ['_'.join(map(str, vals))
           for vals in df.columns.to_flat_index()]
    df.columns = cols
    return df

In [33]:
(jb2
    .groupby(['country_live', 'age'])
    .mean()
    .unstack()
    .pipe(flatten_cols)
)

TypeError: unsupported operand type(s) for +: 'int' and 'str'